In [ ]:
!pip install pandas matplotlib seaborn

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from statsmodels.tsa.stattools import adfuller

sns.set_context("talk")

def load_sensitivity_data(_ngen, _nload, _vm_nelements, _cm_nelements, _sens_path, _hours):
    """Load Sensitivity Matrix"""
    data = {
        'VMP_gen': np.zeros((_hours, _ngen, _vm_nelements)),
        'VMQ_gen': np.zeros((_hours, _ngen, _vm_nelements)),
        'VMP_load': np.zeros((_hours, _nload, _vm_nelements)),
        'VMQ_load': np.zeros((_hours, _nload, _vm_nelements)),
        'CMdSdP_gen': np.zeros((_hours, _ngen, _cm_nelements)),
        'CMdSdQ_gen': np.zeros((_hours, _ngen, _cm_nelements)),
        'CMdSdP_load': np.zeros((_hours, _nload, _cm_nelements)),
        'CMdSdQ_load': np.zeros((_hours, _nload, _cm_nelements))
    }

    for _hour_sim in tqdm(range(_hours)):
        hour_str = f"h{_hour_sim:02d}"
        for _key in data.keys():
            file = f'H{_key.split("_")[0]}_FSP{_key.split("_")[1]}_{hour_str}'
            try:
                sens_df = pd.read_csv(os.path.join(_sens_path, file + '.csv'), index_col=0)
                data[_key][_hour_sim] = sens_df.values
            except FileNotFoundError:
                print(f"File non trovato: {file}")
                continue
    return data


if __name__ == "__main__":
    SENS_PATH = './SensitivityMatrixResults/'
    OUTPUT_PATH = './SensitivityAnalysis/'
    os.makedirs(OUTPUT_PATH, exist_ok=True)

    GEN_NODES = 6
    LOAD_NODES = 16

    CM_ELEMENTS = [
        'Line_0_1', 'Line_1_2', 'Line_2_3', 'Line_3_4',
        'Line_4_5', 'Line_5_6', 'Line_6_7', 'Line_7_8',
        'Line_6_9', 'Line_9_10', 'Line_10_11', 'Line_11_12',
        'Line_12_13', 'Line_11_14', 'Line_14_15', 'Line_15_16',
        'trafo_0_1'
    ]
    VM_ELEMENTS = [f'Bus_{i}' for i in range(17)]
    VM_NETWORK_ELEMENTS = len(VM_ELEMENTS)
    CM_NETWORK_ELEMENTS = len(CM_ELEMENTS)

    HOURS = 8760

    RESOURCES = ['gen', 'load']
    COEFFICIENTS = ['VM', 'CMdSd']
    PRODUCTS = ['P', 'Q']

    sens_data = load_sensitivity_data(GEN_NODES, LOAD_NODES, VM_NETWORK_ELEMENTS, CM_NETWORK_ELEMENTS, SENS_PATH, HOURS)

    # 1. Boxplot per agent (sgen/load) vs network element (lines and trafos/buses)
    for agent_type in RESOURCES:
        for coeff_type in COEFFICIENTS:
            for pq in PRODUCTS:
                key = f"{coeff_type}{pq}_{agent_type}"
                title = f"Sensitivity {coeff_type}{pq} ({agent_type})"
                elements = VM_ELEMENTS if coeff_type == 'VM' else CM_ELEMENTS

                plot_data = []
                for agent in range(GEN_NODES if agent_type == 'gen' else LOAD_NODES):
                    for element_idx, element_name in enumerate(elements):
                        hourly_values = sens_data[key][:, agent, element_idx]
                        for hour, value in enumerate(hourly_values):
                            plot_data.append({
                                'Agent': f"{agent_type}_{agent + 1}",
                                'Element': element_name,
                                'Hour': hour,
                                'Sensitivity': value,
                                'Type': 'Voltage' if coeff_type == 'VM' else 'Congestion'
                            })

                agent_df = pd.DataFrame(plot_data)

                plt.figure(figsize=(20, 10))
                sns.boxplot(x='Element', y='Sensitivity', hue='Agent', data=agent_df)
                plt.title(title)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(os.path.join(OUTPUT_PATH, f'boxplot_{key}.png'))
                plt.close()

    # 2.1 Boxplot VM per network element (lines and trafos/buses) vs agent (sgen/load)
    for element_idx, element_name in enumerate(VM_ELEMENTS):
        for coeff_type in COEFFICIENTS:
            for pq in PRODUCTS:
                gen_key = f"{coeff_type}{pq}_gen"
                load_key = f"{coeff_type}{pq}_load"

                plot_data = []

                for agent in range(GEN_NODES):
                    hourly_values = sens_data[gen_key][:, agent, element_idx]
                    for hour, value in enumerate(hourly_values):
                        plot_data.append({
                            'Agent': f"gen_{agent + 1}",
                            'Hour': hour,
                            'Sensitivity': value,
                            'Type': 'Generator'
                        })

                for agent in range(LOAD_NODES):
                    hourly_values = sens_data[load_key][:, agent, element_idx]
                    for hour, value in enumerate(hourly_values):
                        plot_data.append({
                            'Agent': f"load_{agent + 1}",
                            'Hour': hour,
                            'Sensitivity': value,
                            'Type': 'Load'
                        })

                element_df = pd.DataFrame(plot_data)

                plt.figure(figsize=(20, 10))
                sns.boxplot(x='Agent', y='Sensitivity', hue='Type', data=element_df)
                plt.title(f"VM Sensitivity {coeff_type}{pq} for Network Element {element_name}")
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(os.path.join(OUTPUT_PATH, f'boxplot_VM_element_{element_name}_{coeff_type}{pq}.png'))
                plt.close()

    # 2.2 Boxplot CM per network element (lines and trafos/buses) vs agent (sgen/load)
    for element_idx, element_name in enumerate(CM_ELEMENTS):
        for coeff_type in COEFFICIENTS:
            for pq in PRODUCTS:
                gen_key = f"{coeff_type}{pq}_gen"
                load_key = f"{coeff_type}{pq}_load"

                plot_data = []

                for agent in range(GEN_NODES):
                    hourly_values = sens_data[gen_key][:, agent, element_idx]
                    for hour, value in enumerate(hourly_values):
                        plot_data.append({
                            'Agent': f"gen_{agent + 1}",
                            'Hour': hour,
                            'Sensitivity': value,
                            'Type': 'Generator'
                        })

                for agent in range(LOAD_NODES):
                    hourly_values = sens_data[load_key][:, agent, element_idx]
                    for hour, value in enumerate(hourly_values):
                        plot_data.append({
                            'Agent': f"load_{agent + 1}",
                            'Hour': hour,
                            'Sensitivity': value,
                            'Type': 'Load'
                        })

                element_df = pd.DataFrame(plot_data)

                plt.figure(figsize=(20, 10))
                sns.boxplot(x='Agent', y='Sensitivity', hue='Type', data=element_df)
                plt.title(f"CM Sensitivity {coeff_type}{pq} for Network Element {element_name}")
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(os.path.join(OUTPUT_PATH, f'boxplot_CM_element_{element_name}_{coeff_type}{pq}.png'))
                plt.close()

    # 3.1 Cumulative Distribution Function VM (CDF) per network element
    for element_idx, element_name in enumerate(VM_ELEMENTS):
        plt.figure(figsize=(15, 8))

        for pq in PRODUCTS:
            gen_key = f"VM{pq}_gen"
            load_key = f"VM{pq}_load"

            for agent in range(GEN_NODES):
                values = sens_data[gen_key][:, agent, element_idx]
                sorted_data = np.sort(values)
                cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
                plt.plot(sorted_data, cdf,
                         label=f"VM{pq} gen_{agent + 1}",
                         linestyle='--')

            for agent in range(LOAD_NODES):
                values = sens_data[load_key][:, agent, element_idx]
                sorted_data = np.sort(values)
                cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
                plt.plot(sorted_data, cdf,
                         label=f"VM{pq} load_{agent + 1}",
                         linestyle='-')

        plt.title(f"CDF VM for Network Element {element_name}")
        plt.xlabel('Sensitivity Coefficient')
        plt.ylabel('CDF')
        plt.grid(True)
        plt.legend(loc='upper left')
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_PATH, f'cdf_VM_element_{element_name}.png'))
        plt.close()

    # 3.2 Cumulative Distribution Function CM (CDF) per network element
    for element_idx, element_name in enumerate(CM_ELEMENTS):
        plt.figure(figsize=(15, 8))

        for pq in PRODUCTS:
            gen_key = f"CMdSd{pq}_gen"
            load_key = f"CMdSd{pq}_load"

            for agent in range(GEN_NODES):
                values = sens_data[gen_key][:, agent, element_idx]
                sorted_data = np.sort(values)
                cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
                plt.plot(sorted_data, cdf,
                         label=f"CMdSd{pq} gen_{agent + 1}",
                         linestyle='--')

            for agent in range(LOAD_NODES):
                values = sens_data[load_key][:, agent, element_idx]
                sorted_data = np.sort(values)
                cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
                plt.plot(sorted_data, cdf,
                         label=f"CMdSd{pq} load_{agent + 1}",
                         linestyle='-')

        plt.title(f"CDF VM for Network Element {element_name}")
        plt.xlabel('Sensitivity Coefficient')
        plt.ylabel('CDF')
        plt.grid(True)
        plt.legend(loc='upper left')
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_PATH, f'cdf_CM_element_{element_name}.png'))
        plt.close()

    # 4 Heatmap of medians Elements
    for _coeff_type in COEFFICIENTS:
        for pq in PRODUCTS:
            gen_key = f"{_coeff_type}{pq}_gen"
            load_key = f"{_coeff_type}{pq}_load"

            # Evaluate the median per each combination agent-network element
            if _coeff_type == 'VM':
                median_data = np.zeros((GEN_NODES + LOAD_NODES, VM_NETWORK_ELEMENTS))
                _elements = VM_ELEMENTS
                _what = 'VM'
            else:
                median_data = np.zeros((GEN_NODES + LOAD_NODES, CM_NETWORK_ELEMENTS))
                _elements = CM_ELEMENTS
                _what = 'CM'

            for agent in range(GEN_NODES):
                median_data[agent, :] = np.median(sens_data[gen_key][:, agent, :], axis=0)

            for agent in range(LOAD_NODES):
                median_data[GEN_NODES + agent, :] = np.median(sens_data[load_key][:, agent, :], axis=0)

            plt.figure(figsize=(16, 12))
            sns.heatmap(median_data,
                        cmap='coolwarm',
                        annot=True, fmt=".2f",
                        xticklabels=[f"{i}" for i in _elements],
                        yticklabels=[f"Gen {i + 1}" for i in range(GEN_NODES)] +
                                    [f"Load {i + 1}" for i in range(LOAD_NODES)])
            plt.title(f"Median {_coeff_type}{pq} Sensitivity Coefficients")
            plt.tight_layout()
            plt.savefig(os.path.join(OUTPUT_PATH, f'heatmap_{_what}_{_coeff_type}{pq}.png'))
            plt.close()

    # 5. Prepared Combined DataFrame
    rows = []
    for _hour in tqdm(range(HOURS)):
        for element_idx, element_name in enumerate(VM_ELEMENTS):
            row = {
                'hour': _hour,
                'element': element_idx,
                'date': pd.Timestamp('2023-01-01') + pd.Timedelta(hours=_hour)
            }

            # Add average sensibility per network element
            for pq in PRODUCTS:
                # Mean between all generators
                row[f'VM{pq}_gen_mean'] = np.mean([sens_data[f'VM{pq}_gen'][_hour, _sgen, element_idx] for _sgen in range(GEN_NODES)])
                # Mean between all loads
                row[f'VM{pq}_load_mean'] = np.mean([sens_data[f'VM{pq}_load'][_hour, _load, element_idx] for _load in range(LOAD_NODES)])
            rows.append(row)
    combined_VM_df = pd.DataFrame(rows)

    rows = []
    for _hour in tqdm(range(HOURS)):
        for element_idx, element_name in enumerate(CM_ELEMENTS):
            row = {
                'hour': _hour,
                'element': element_idx,
                'date': pd.Timestamp('2023-01-01') + pd.Timedelta(hours=_hour)
            }

            # Add average sensibility per network element
            for _coeff_type in COEFFICIENTS:
                for pq in PRODUCTS:
                    # Mean between all generators
                    row[f'CMdSd{pq}_gen_mean'] = np.mean([sens_data[f'CMdSd{pq}_gen'][_hour, _sgen, element_idx] for _sgen in range(GEN_NODES)])
                    # Mean between all loads
                    row[f'CMdSd{pq}_load_mean'] = np.mean([sens_data[f'CMdSd{pq}_load'][_hour, _load, element_idx] for _load in range(LOAD_NODES)])
                rows.append(row)
    combined_CM_df = pd.DataFrame(rows)

    # 6. Temporal Analysis
    # Evaluate daily averages
    daily_avg_VM = combined_VM_df.groupby(combined_VM_df['date'].dt.hour).mean()
    daily_avg_CM = combined_CM_df.groupby(combined_CM_df['date'].dt.hour).mean()

    # Plot per ogni tipo di coefficiente
    for _coeff_type in COEFFICIENTS:
        if _coeff_type == 'VM':
            _what = 'VM'
            daily_avg = daily_avg_VM
        else:
            _what = 'CM'
            daily_avg = daily_avg_CM

        for pq in PRODUCTS:
            plt.figure(figsize=(12, 6))

            for _agent in RESOURCES:
                plt.plot(daily_avg.index,
                         daily_avg[f'{_coeff_type}{pq}_{_agent}_mean'],
                         label=f'{_coeff_type}{pq} {_agent}')

            plt.title(f'Daily Trend {_what}: {_coeff_type}{pq} Coefficients')
            plt.xlabel('Hour of Day')
            plt.ylabel('Sensitivity Value')
            plt.legend()
            plt.grid(True)
            plt.savefig(os.path.join(OUTPUT_PATH, f'trend_{_what}_{_coeff_type}{pq}.png'))
            plt.close()

    # 7. Coefficient Correlation
    corr_cols_VM = [c for c in combined_VM_df.columns if '_mean' in c]
    corr_matrix_VM = combined_VM_df[corr_cols_VM].corr()
    corr_cols_CM = [c for c in combined_CM_df.columns if '_mean' in c]
    corr_matrix_CM = combined_CM_df[corr_cols_CM].corr()

    # Correlation Heatmap
    plt.figure(figsize=(14, 12))
    sns.heatmap(corr_matrix_VM,
                annot=True,
                cmap='coolwarm',
                fmt=".2f",
                center=0)
    plt.title('Correlation Matrix Between Sensitivity Coefficients VM')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_PATH, 'correlation_matrix_VM.png'))
    plt.close()

    plt.figure(figsize=(14, 12))
    sns.heatmap(corr_matrix_CM,
                annot=True,
                cmap='coolwarm',
                fmt=".2f",
                center=0)
    plt.title('Correlation Matrix Between Sensitivity Coefficients CM')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_PATH, 'correlation_matrix_CM.png'))
    plt.close()

    # 8. Evaluate Stationary
    # A timeseries is considered stationary if its statistical properties (mean, variance, autocorrelation)
    # do not change over time. In order to evaluate this property we conduct the Augmented Dickey-Fuller (ADF) test:
    # - Null Hypothesis: The series has a unit root (non-stationary)
    # - p-value < 0.05: Reject the null hypothesis (series is stationary)

    stationarity_results_VM = []
    for coeff in ['VMP_gen_mean', 'VMQ_gen_mean', 'VMP_load_mean', 'VMQ_load_mean']:
        result = adfuller(combined_VM_df[coeff].dropna())
        stationarity_results_VM.append({
            'coefficient': coeff,
            'ADF Statistic': result[0],
            'p-value': result[1],
            'is_stationary': result[1] < 0.05
        })

    stationarity_df_VM = pd.DataFrame(stationarity_results_VM)
    print("\nStationarity Results VM:")
    print(stationarity_df_VM.to_string(index=False))

    stationarity_results_CM = []
    for coeff in ['CMdSdP_gen_mean', 'CMdSdQ_gen_mean', 'CMdSdP_load_mean', 'CMdSdQ_load_mean']:
        result = adfuller(combined_CM_df[coeff].dropna())
        stationarity_results_CM.append({
            'coefficient': coeff,
            'ADF Statistic': result[0],
            'p-value': result[1],
            'is_stationary': result[1] < 0.05
        })

    stationarity_df_CM = pd.DataFrame(stationarity_results_CM)
    print("\nStationarity Results CM:")
    print(stationarity_df_CM.to_string(index=False))


In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from statsmodels.tsa.stattools import adfuller

sns.set_context("talk") 

def load_sensitivity_data(_ngen, _nload, _vm_nelements, _cm_nelements, _sens_path, _hours):
    """Load Sensitivity Matrix"""
    data = {
        'VMP_gen': np.zeros((_hours, _ngen, _vm_nelements)),
        'VMQ_gen': np.zeros((_hours, _ngen, _vm_nelements)),
        'VMP_load': np.zeros((_hours, _nload, _vm_nelements)),
        'VMQ_load': np.zeros((_hours, _nload, _vm_nelements)),
        'CMdSdP_gen': np.zeros((_hours, _ngen, _cm_nelements)),
        'CMdSdQ_gen': np.zeros((_hours, _ngen, _cm_nelements)),
        'CMdSdP_load': np.zeros((_hours, _nload, _cm_nelements)),
        'CMdSdQ_load': np.zeros((_hours, _nload, _cm_nelements))
    }

    for _hour_sim in tqdm(range(_hours)):
        hour_str = f"h{_hour_sim:02d}"
        for _key in data.keys():
            file = f'H{_key.split("_")[0]}_FSP{_key.split("_")[1]}_{hour_str}'
            try:
                sens_df = pd.read_csv(os.path.join(_sens_path, file + '.csv'), index_col=0)
                data[_key][_hour_sim] = sens_df.values
            except FileNotFoundError:
                print(f"File non trovato: {file}")
                continue
    return data


if __name__ == "__main__":
    SENS_PATH = './SensitivityMatrixResults_v3/'
    OUTPUT_PATH = './SensitivityAnalysis_v3/'
    os.makedirs(OUTPUT_PATH, exist_ok=True)

    GEN_NODES = 16
    LOAD_NODES = 16

    CM_ELEMENTS = [
        'Line_0_1', 'Line_1_2', 'Line_2_3', 'Line_3_4',
        'Line_4_5', 'Line_5_6', 'Line_6_7', 'Line_7_8',
        'Line_5_9', 'Line_0_16', 'Line_15_16', 'Line_10_15',
        'Line_14_15', 'Line_11_14', 'Line_11_12', 'Line_12_13',
        'trafo_0_1'
    ]
    VM_ELEMENTS = [f'Bus_{i}' for i in range(17)]
    VM_NETWORK_ELEMENTS = len(VM_ELEMENTS)
    CM_NETWORK_ELEMENTS = len(CM_ELEMENTS)

    HOURS = 7158

    RESOURCES = ['gen', 'load']
    COEFFICIENTS = ['VM', 'CMdSd']
    PRODUCTS = ['P', 'Q']

    sens_data = load_sensitivity_data(GEN_NODES, LOAD_NODES, VM_NETWORK_ELEMENTS, CM_NETWORK_ELEMENTS, SENS_PATH, HOURS)


    # 1. Boxplot per agent (sgen/load) vs network element (lines and trafos/buses)
    for agent_type in RESOURCES:
        for coeff_type in COEFFICIENTS:
            for pq in PRODUCTS:
                key = f"{coeff_type}{pq}_{agent_type}"
                title = f"Sensitivity {coeff_type}{pq} ({agent_type})"
                elements = VM_ELEMENTS if coeff_type == 'VM' else CM_ELEMENTS

                plot_data = []
                for agent in range(GEN_NODES if agent_type == 'gen' else LOAD_NODES):
                    for element_idx, element_name in enumerate(elements):
                        hourly_values = sens_data[key][:, agent, element_idx]
                        for hour, value in enumerate(hourly_values):
                            plot_data.append({
                                'Agent': f"{agent_type}_{agent + 1}",
                                'Element': element_name,
                                'Hour': hour,
                                'Sensitivity': value,
                                'Type': 'Voltage' if coeff_type == 'VM' else 'Congestion'
                            })

                agent_df = pd.DataFrame(plot_data)

                plt.figure(figsize=(20, 10))
                sns.boxplot(x='Element', y='Sensitivity', hue='Agent', data=agent_df)
                plt.title(title)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(os.path.join(OUTPUT_PATH, f'boxplot_{key}.png'))
                plt.close()

    # 2.1 Boxplot VM per network element (lines and trafos/buses) vs agent (sgen/load)
    for element_idx, element_name in enumerate(VM_ELEMENTS):
        for coeff_type in COEFFICIENTS:
            for pq in PRODUCTS:
                gen_key = f"{coeff_type}{pq}_gen"
                load_key = f"{coeff_type}{pq}_load"

                plot_data = []

                for agent in range(GEN_NODES):
                    hourly_values = sens_data[gen_key][:, agent, element_idx]
                    for hour, value in enumerate(hourly_values):
                        plot_data.append({
                            'Agent': f"gen_{agent + 1}",
                            'Hour': hour,
                            'Sensitivity': value,
                            'Type': 'Generator'
                        })

                for agent in range(LOAD_NODES):
                    hourly_values = sens_data[load_key][:, agent, element_idx]
                    for hour, value in enumerate(hourly_values):
                        plot_data.append({
                            'Agent': f"load_{agent + 1}",
                            'Hour': hour,
                            'Sensitivity': value,
                            'Type': 'Load'
                        })

                element_df = pd.DataFrame(plot_data)

                plt.figure(figsize=(20, 10))
                sns.boxplot(x='Agent', y='Sensitivity', hue='Type', data=element_df)
                plt.title(f"VM Sensitivity {coeff_type}{pq} for Network Element {element_name}")
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(os.path.join(OUTPUT_PATH, f'boxplot_VM_element_{element_name}_{coeff_type}{pq}.png'))
                plt.close()

    # 2.2 Boxplot CM per network element (lines and trafos/buses) vs agent (sgen/load)
    for element_idx, element_name in enumerate(CM_ELEMENTS):
        for coeff_type in COEFFICIENTS:
            for pq in PRODUCTS:
                gen_key = f"{coeff_type}{pq}_gen"
                load_key = f"{coeff_type}{pq}_load"

                plot_data = []

                for agent in range(GEN_NODES):
                    hourly_values = sens_data[gen_key][:, agent, element_idx]
                    for hour, value in enumerate(hourly_values):
                        plot_data.append({
                            'Agent': f"gen_{agent + 1}",
                            'Hour': hour,
                            'Sensitivity': value,
                            'Type': 'Generator'
                        })

                for agent in range(LOAD_NODES):
                    hourly_values = sens_data[load_key][:, agent, element_idx]
                    for hour, value in enumerate(hourly_values):
                        plot_data.append({
                            'Agent': f"load_{agent + 1}",
                            'Hour': hour,
                            'Sensitivity': value,
                            'Type': 'Load'
                        })

                element_df = pd.DataFrame(plot_data)

                plt.figure(figsize=(20, 10))
                sns.boxplot(x='Agent', y='Sensitivity', hue='Type', data=element_df)
                plt.title(f"CM Sensitivity {coeff_type}{pq} for Network Element {element_name}")
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(os.path.join(OUTPUT_PATH, f'boxplot_CM_element_{element_name}_{coeff_type}{pq}.png'))
                plt.close()

    # 3.1 Cumulative Distribution Function VM (CDF) per network element
    for element_idx, element_name in enumerate(VM_ELEMENTS):
        plt.figure(figsize=(15, 8))

        for pq in PRODUCTS:
            gen_key = f"VM{pq}_gen"
            load_key = f"VM{pq}_load"

            for agent in range(GEN_NODES):
                values = sens_data[gen_key][:, agent, element_idx]
                sorted_data = np.sort(values)
                cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
                plt.plot(sorted_data, cdf,
                         label=f"VM{pq} gen_{agent + 1}",
                         linestyle='--')

            for agent in range(LOAD_NODES):
                values = sens_data[load_key][:, agent, element_idx]
                sorted_data = np.sort(values)
                cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
                plt.plot(sorted_data, cdf,
                         label=f"VM{pq} load_{agent + 1}",
                         linestyle='-')
        
        plt.title(f"CDF VM for Network Element {element_name}")
        plt.xlabel('Sensitivity Coefficient')
        plt.ylabel('CDF')
        plt.grid(True)
        plt.legend(loc='upper left')
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_PATH, f'cdf_VM_element_{element_name}.png'))
        plt.close()

    # 3.2 Cumulative Distribution Function CM (CDF) per network element
    for element_idx, element_name in enumerate(CM_ELEMENTS):
        plt.figure(figsize=(15, 8))

        for pq in PRODUCTS:
            gen_key = f"CMdSd{pq}_gen"
            load_key = f"CMdSd{pq}_load"

            for agent in range(GEN_NODES):
                values = sens_data[gen_key][:, agent, element_idx]
                sorted_data = np.sort(values)
                cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
                plt.plot(sorted_data, cdf,
                         label=f"CMdSd{pq} gen_{agent + 1}",
                         linestyle='--')

            for agent in range(LOAD_NODES):
                values = sens_data[load_key][:, agent, element_idx]
                sorted_data = np.sort(values)
                cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
                plt.plot(sorted_data, cdf,
                         label=f"CMdSd{pq} load_{agent + 1}",
                         linestyle='-')

        plt.title(f"CDF VM for Network Element {element_name}")
        plt.xlabel('Sensitivity Coefficient')
        plt.ylabel('CDF')
        plt.grid(True)
        plt.legend(loc='upper left')
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_PATH, f'cdf_CM_element_{element_name}.png'))
        plt.close()

    # 4 Heatmap of medians Elements
    for _coeff_type in COEFFICIENTS:
        for pq in PRODUCTS:
            gen_key = f"{_coeff_type}{pq}_gen"
            load_key = f"{_coeff_type}{pq}_load"

            # Evaluate the median per each combination agent-network element
            if _coeff_type == 'VM':
                median_data = np.zeros((GEN_NODES + LOAD_NODES, VM_NETWORK_ELEMENTS))
                _elements = VM_ELEMENTS
                _what = 'VM'
            else:
                median_data = np.zeros((GEN_NODES + LOAD_NODES, CM_NETWORK_ELEMENTS))
                _elements = CM_ELEMENTS
                _what = 'CM'

            for agent in range(GEN_NODES):
                median_data[agent, :] = np.median(sens_data[gen_key][:, agent, :], axis=0)

            for agent in range(LOAD_NODES):
                median_data[GEN_NODES + agent, :] = np.median(sens_data[load_key][:, agent, :], axis=0)

            plt.figure(figsize=(16, 12))
            sns.heatmap(median_data,
                        cmap='coolwarm',
                        annot=True, fmt=".2f",
                        xticklabels=[f"{i}" for i in _elements],
                        yticklabels=[f"Gen {i + 1}" for i in range(GEN_NODES)] +
                                    [f"Load {i + 1}" for i in range(LOAD_NODES)])
            plt.title(f"Median {_coeff_type}{pq} Sensitivity Coefficients")
            plt.tight_layout()
            plt.savefig(os.path.join(OUTPUT_PATH, f'heatmap_{_what}_{_coeff_type}{pq}.png'))
            plt.close()

    # 5. Prepared Combined DataFrame
    rows = []
    for _hour in tqdm(range(HOURS)):
        for element_idx, element_name in enumerate(VM_ELEMENTS):
            row = {
                'hour': _hour,
                'element': element_idx,
                'date': pd.Timestamp('2023-01-01') + pd.Timedelta(hours=_hour)
            }

            # Add average sensibility per network element
            for pq in PRODUCTS:
                # Mean between all generators
                row[f'VM{pq}_gen_mean'] = np.mean([sens_data[f'VM{pq}_gen'][_hour, _sgen, element_idx] for _sgen in range(GEN_NODES)])
                # Mean between all loads
                row[f'VM{pq}_load_mean'] = np.mean([sens_data[f'VM{pq}_load'][_hour, _load, element_idx] for _load in range(LOAD_NODES)])
            rows.append(row)
    combined_VM_df = pd.DataFrame(rows)

    rows = []
    for _hour in tqdm(range(HOURS)):
        for element_idx, element_name in enumerate(CM_ELEMENTS):
            row = {
                'hour': _hour,
                'element': element_idx,
                'date': pd.Timestamp('2023-01-01') + pd.Timedelta(hours=_hour)
            }

            # Add average sensibility per network element
            for _coeff_type in COEFFICIENTS:
                for pq in PRODUCTS:
                    # Mean between all generators
                    row[f'CMdSd{pq}_gen_mean'] = np.mean([sens_data[f'CMdSd{pq}_gen'][_hour, _sgen, element_idx] for _sgen in range(GEN_NODES)])
                    # Mean between all loads
                    row[f'CMdSd{pq}_load_mean'] = np.mean([sens_data[f'CMdSd{pq}_load'][_hour, _load, element_idx] for _load in range(LOAD_NODES)])
                rows.append(row)
    combined_CM_df = pd.DataFrame(rows)

    # 6. Temporal Analysis
    # Evaluate daily averages
    daily_avg_VM = combined_VM_df.groupby(combined_VM_df['date'].dt.hour).mean()
    daily_avg_CM = combined_CM_df.groupby(combined_CM_df['date'].dt.hour).mean()

    # Plot per ogni tipo di coefficiente
    for _coeff_type in COEFFICIENTS:
        if _coeff_type == 'VM':
            _what = 'VM'
            daily_avg = daily_avg_VM
        else:
            _what = 'CM'
            daily_avg = daily_avg_CM

        for pq in PRODUCTS:
            plt.figure(figsize=(12, 6))

            for _agent in RESOURCES:
                plt.plot(daily_avg.index,
                         daily_avg[f'{_coeff_type}{pq}_{_agent}_mean'],
                         label=f'{_coeff_type}{pq} {_agent}')

            plt.title(f'Daily Trend {_what}: {_coeff_type}{pq} Coefficients')
            plt.xlabel('Hour of Day')
            plt.ylabel('Sensitivity Value')
            plt.legend()
            plt.grid(True)
            plt.savefig(os.path.join(OUTPUT_PATH, f'trend_{_what}_{_coeff_type}{pq}.png'))
            plt.close()

    # 7. Coefficient Correlation
    corr_cols_VM = [c for c in combined_VM_df.columns if '_mean' in c]
    corr_matrix_VM = combined_VM_df[corr_cols_VM].corr()
    corr_cols_CM = [c for c in combined_CM_df.columns if '_mean' in c]
    corr_matrix_CM = combined_CM_df[corr_cols_CM].corr()

    # Correlation Heatmap
    plt.figure(figsize=(14, 12))
    sns.heatmap(corr_matrix_VM,
                annot=True,
                cmap='coolwarm',
                fmt=".2f",
                center=0)
    plt.title('Correlation Matrix Between Sensitivity Coefficients VM')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_PATH, 'correlation_matrix_VM.png'))
    plt.close()

    plt.figure(figsize=(14, 12))
    sns.heatmap(corr_matrix_CM,
                annot=True,
                cmap='coolwarm',
                fmt=".2f",
                center=0)
    plt.title('Correlation Matrix Between Sensitivity Coefficients CM')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_PATH, 'correlation_matrix_CM.png'))
    plt.close()

    # 8. Evaluate Stationary
    # A timeseries is considered stationary if its statistical properties (mean, variance, autocorrelation)
    # do not change over time. In order to evaluate this property we conduct the Augmented Dickey-Fuller (ADF) test:
    # - Null Hypothesis: The series has a unit root (non-stationary)
    # - p-value < 0.05: Reject the null hypothesis (series is stationary)

    stationarity_results_VM = []
    for coeff in ['VMP_gen_mean', 'VMQ_gen_mean', 'VMP_load_mean', 'VMQ_load_mean']:
        result = adfuller(combined_VM_df[coeff].dropna())
        stationarity_results_VM.append({
            'coefficient': coeff,
            'ADF Statistic': result[0],
            'p-value': result[1],
            'is_stationary': result[1] < 0.05
        })

    stationarity_df_VM = pd.DataFrame(stationarity_results_VM)
    print("\nStationarity Results VM:")
    print(stationarity_df_VM.to_string(index=False))

    stationarity_results_CM = []
    for coeff in ['CMdSdP_gen_mean', 'CMdSdQ_gen_mean', 'CMdSdP_load_mean', 'CMdSdQ_load_mean']:
        result = adfuller(combined_CM_df[coeff].dropna())
        stationarity_results_CM.append({
            'coefficient': coeff,
            'ADF Statistic': result[0],
            'p-value': result[1],
            'is_stationary': result[1] < 0.05
        })

    stationarity_df_CM = pd.DataFrame(stationarity_results_CM)
    print("\nStationarity Results CM:")
    print(stationarity_df_CM.to_string(index=False))